In this notebook we will:
- Use Albumentations to create a new augmented datasets with thousands of augmented images
- Fine tune the EfficientNetV2 model for maximum F1 score and the final deployment of the model to our website

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import albumentations as A

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Hyperparameters
IMAGE_SIZE = 256
BATCH_SIZE = 8
CHANNELS= 3
ES_EPOCHS = 20 
CLASSES = 10
EPOCHS= 200

In [4]:
CLASS_NAMES = ["1989", "Acoustic", "Fearless", "Folkmore", "Lover", "Midnights", "Red", "Reputation", "Speak Now", "TTPD"]

In [5]:
# Train
train_directory = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/train"

# Val
val_directory = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/val"

# Test
test_directory = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/test"

Data augmentation pipeline using Albumentations 

In [6]:
import random

import cv2
from matplotlib import pyplot as plt

In [7]:
def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)

In [ ]:
image = cv2.imread('Images_App_Test/1989_orange.jpeg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
visualize(image)

In [9]:
transforms = A.Compose([
            A.Rotate(limit=40),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
            A.HorizontalFlip(),
            A.ShiftScaleRotate(p=0.5),
            A.Blur(blur_limit=3),
            A.OpticalDistortion(),
            A.GridDistortion(),
        ])

In [16]:
import os
import cv2
import random
import albumentations as A

# Define paths
dataset_path = 'train'  # Replace with your dataset path
augmented_dataset_path = 'augmented_train'  # Replace with desired output path

# Define augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),              # Horizontal flip with 50% probability
    A.VerticalFlip(p=0.5),                # Vertical flip with 50% probability
    A.RandomRotate90(p=0.5),              # Random 90-degree rotation with 50% probability
    A.RandomBrightnessContrast(p=0.5),      # Random brightness/contrast adjustments
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.5),  # Random shift, scale, and rotation
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),  # Random hue, saturation, and value adjustments
    A.GaussianBlur(blur_limit=(3, 7), p=0.5),  # Gaussian blur with random kernel size
], p=1.0)  # Apply the entire pipeline with probability 1.0

# Function to augment images in a directory
def augment_images(source_dir, dest_dir, num_augmentations=10):
    """
    Augments images in a source directory and saves them to a destination directory.

    Args:
        source_dir (str): Path to the directory containing the original images.
        dest_dir (str): Path to the directory where augmented images will be saved.
        num_augmentations (int): Number of augmentations to generate per image.
    """
    for filename in os.listdir(source_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(source_dir, filename)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            for i in range(num_augmentations):
                augmented_image = transform(image=image)['image']
                augmented_filename = f"{os.path.splitext(filename)[0]}_aug_{i}.jpg"
                augmented_path = os.path.join(dest_dir, augmented_filename)
                cv2.imwrite(augmented_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

# Create the augmented dataset directory structure
if not os.path.exists(augmented_dataset_path):
    os.makedirs(augmented_dataset_path)

# Iterate through classes and augment images
for class_name in os.listdir(dataset_path):
    class_source_dir = os.path.join(dataset_path, class_name)
    class_dest_dir = os.path.join(augmented_dataset_path, class_name)
    if not os.path.exists(class_dest_dir):
        os.makedirs(class_dest_dir)
    augment_images(class_source_dir, class_dest_dir)

print("Data augmentation complete!")

Data augmentation complete!
